# Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the training set

In [2]:
dataset_train = pd.read_csv('Train.csv')
training_set = dataset_train.iloc[:, 1:].values


# Feature Scaling

In [3]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
training_set_scaled = sc.fit_transform(training_set)

# Creating a data structure with 870 timesteps and 1 output

In [4]:
X_train = []
y_train = []
for i in range(1170, len(training_set)):
  X_train.append(training_set_scaled[i-1170:i,:])
  y_train.append(training_set_scaled[i, 1])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping

In [5]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 5))

# Importing the keras libraries and packages

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


# Initializing the RNN

In [7]:
regressor = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

In [8]:
regressor.add(LSTM(units = 128, return_sequences = True, input_shape = (X_train.shape[1], 5)))
regressor.add(Dropout(0.2))

#Adding the second LSTM layer and some Dropout regularisation

In [9]:
regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding the third LSTM layer and some Dropout regularisation

In [10]:
regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.2))

#Adding the fourth LSTM layer and some Dropout regularisation

In [11]:
regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.2))

#Adding the fifth LSTM layer and some Dropout regularisation

In [12]:
regressor.add(LSTM(units = 128))
regressor.add(Dropout(0.2))

#The output layer

In [13]:
regressor.add(Dense(units = 1))

#Compiling the RNN

In [ ]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set

In [ ]:
regressor.fit(X_train, y_train, batch_size = 32, epochs = 200)

# Getting the real stock price of 22/7/2020

In [ ]:
dataset_test = pd.read_csv('Test.csv')
real_open_price = dataset_test.iloc[:, 1].values

# Getting the predicted stock prices of 22/7/2020

In [ ]:
dataset_total = pd.concat((dataset_train['Close', 'Open', 'High', 'Low', 'Volume'], dataset_test['Close', 'Open', 'High', 'Low', 'Volume']), axis = 0)
inputs = dataset_total[len(dataset_total)-len(dataset_test)-1170:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(1170, 1560):
  X_test.append(inputs[i-1170:i,:])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 5))
predicted_open_prices = regressor.predict(X_test)
predicted_open_prices = sc.inverse_transform(predicted_open_prices)